In [85]:
from google.cloud import vision

In [86]:
import os

In [87]:
import requests

In [88]:
import pandas as pd

In [89]:
import gspread

In [90]:
from oauth2client.service_account import ServiceAccountCredentials

In [91]:
image_csv = "/Users/jeslyvarghese/Downloads/lux_photos.csv"

In [92]:
data = pd.read_csv(image_csv, encoding='utf-8')

In [93]:
print(data.keys())

Index(['id', 'id_inventairecna', 'id_originel', 'titre_propre', 'titre_forge',
       'titre_de_serie', 'description_de_limage', 'photographe',
       'date_de_prise_de_vue', 'couleur', 'type_de_support_original',
       'denomination_du_procede_technique', 'vertical',
       'format_de_limage_originale', 'format_du_support_original', 'copyright',
       'media', 'Lieu_de_prise_de_vue', 'status', 'categories'],
      dtype='object')


In [94]:
image_urls = [os.path.join(
    'https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/', d+".jpg"
) for d in data['id_inventairecna']]

In [12]:
service_account = "/Users/jeslyvarghese/Workspace/Work/Teliportme/teliport-api-server/Website/pyapi/pyapi/service-account.json"

In [97]:
print("\n".join(image_urls))

https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAEL000120Z01_traite.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAEL000106Z01_traite.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_14.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_13.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_12.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_11.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_10.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_09.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_07.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_08_IMG_08.jpg
https://cnahackathon.s3.eu-west-3.amazonaws.com/PhotoResize/HISAFU_DiaBox_07_IMG_19.jpg
https://cnahackathon.s3.eu-west-3.

In [13]:
client = vision.ImageAnnotatorClient.from_service_account_json(service_account)

In [52]:
annotations_dict = {}
for image_url in image_urls[:5]:
    r = client.annotate_image({'image': {'source': {'image_uri': image_url}}})
    annotations_dict[image_url] = r.label_annotations

In [62]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(service_account, scope)

In [63]:
gc = gspread.authorize(credentials)

In [84]:
workbook = gc.open_by_key('1wZ4C8S-MoFZbVXj1mYno78gUnyIXlJRGgGR-WWnNjKs')

In [67]:
print(workbook.worksheets())

[<Worksheet 'content' id:155590555>, <Worksheet 'main-page' id:1353043662>, <Worksheet 'filters' id:1494297336>, <Worksheet 'Upgrade / Help' id:1279231234>, <Worksheet 'sort' id:1487498550>, <Worksheet 'about-page' id:1248649032>]


In [71]:
worksheet = workbook.add_worksheet(title="CNA Photos", rows=5, cols=len(data.keys()))

In [83]:
for idx, image_url in enumerate(annotations_dict.keys()):
    title = data['titre_propre'][idx]
    description = data['description_de_limage'][idx]
    annotations = annotations_dict[image_url]
    old_tags = data['categories'][idx]
    tags = [a.description for a in annotations if a.score > 0.7]
    worksheet.update_cell(idx+2, 1, image_url)
    worksheet.update_cell(idx+2, 2, title)
    worksheet.update_cell(idx+2, 3, description)
    worksheet.update_cell(idx+2, 4, ",".join(tags))
    worksheet.update_cell(idx+2, 5, old_tags)